In [ ]:
from rag.GoogleSearchScrapper import GoogleSearchModule

In [ ]:
# Create an instance of GoogleSearchModule with the topic you want to search for
search_module = GoogleSearchModule("Write a post about the benefits of using a chatbot on a website.")

In [ ]:
# Get URLs, filtered to exclude certain social media sites
filtered_urls = search_module.get_url("Write a post about the benefits of using a chatbot on a website.")

In [ ]:
# Print the filtered URLs
print(filtered_urls)

In [ ]:
from langchain_community.document_loaders import WebBaseLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

In [ ]:
def pretty_print_docs(docs):
    print(
        f"\n{'-' * 100}\n".join(
            [f"Document {i+1}:\n\n" + d.page_content for i, d in enumerate(docs)]
        )
    )

In [ ]:
loaded_documents = []
for url in filtered_urls:
    loader = WebBaseLoader(url)
    loaded_documents += loader.load()

In [ ]:
loaded_documents

In [ ]:
pretty_print_docs(loaded_documents)

In [ ]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=100)
documents = text_splitter.split_documents(loaded_documents)

In [ ]:
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings
from langchain_community.embeddings import HuggingFaceEmbeddings

In [ ]:
embeddings = HuggingFaceEmbeddings()

In [ ]:
db = FAISS.from_documents(documents, embeddings)

In [ ]:
query = "Write a short post on the benefits of using a chatbot on a website."
retriever = db.as_retriever(search_kwargs={"k": 10})

In [ ]:
retriever.invoke(query)

In [ ]:
from langchain.retrievers import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import FlashrankRerank

In [ ]:
compressor = FlashrankRerank()
compression_retriever = ContextualCompressionRetriever(
    base_compressor=compressor, base_retriever=retriever
)

In [ ]:
compressed_docs = compression_retriever.invoke(query)

In [ ]:
compressed_docs

In [ ]:
pretty_print_docs(compressed_docs)

___

In [17]:
import os
from dotenv import load_dotenv
from langchain.schema import HumanMessage
from camel_pipeline import camel_pipeline
from rag import return_filtered_urls, return_camel_response_doc, return_url_doc, ingest_all, retrieval_pipeline

In [2]:
assistant_role_name = "Content Writer"
user_role_name = "Founder and CEO"
user_task = "Write a post about the benefits of using a chatbot on a website."

In [3]:
specified_task, camel_response = camel_pipeline(assistant_role_name, user_role_name, user_task)

2024-06-04 18:00:06,581 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2024-06-04 18:00:08,519 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2024-06-04 18:00:09,414 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2024-06-04 18:00:10,581 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2024-06-04 18:00:11,475 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2024-06-04 18:00:12,339 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2024-06-04 18:00:13,838 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2024-06-04 18:00:15,052 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2024-06-04 18:00:16,258 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "

In [4]:
specified_task

'Write a 500-word blog post highlighting the unique benefits of using a chatbot on a website, focusing on how it can enhance customer engagement, streamline support processes, and drive conversions. Incorporate real-life examples and case studies to illustrate the impact of chatbots in improving user experience and business outcomes.'

In [5]:
camel_response

"Chatbots have revolutionized the way businesses interact with their website visitors, offering a seamless and efficient way to engage, support, and convert customers. They play a crucial role in enhancing user experience and driving business growth.One notable example is Sephora, which implemented a chatbot on their website to provide personalized product recommendations, makeup tutorials, and beauty tips. This interactive approach has significantly increased customer engagement and satisfaction, leading to a boost in sales and brand loyalty.Chatbots streamline support processes by providing instant responses to customer inquiries, resolving common issues, and directing users to relevant resources. This not only reduces wait times but also ensures consistent and accurate support, ultimately enhancing customer satisfaction and loyalty. For example, a chatbot on a banking website can quickly assist customers with account inquiries, saving time for both the customer and the support team.

In [6]:
urls = return_filtered_urls(specified_task)
urls

['https://www.zendesk.com/in/blog/5-benefits-using-ai-bots-customer-service/',
 'https://www.talkdesk.com/blog/chatbots-customer-service/',
 'https://umni.bg/en/blog/10-benefits-of-adding-an-ai-chatbot-on-your-website/']

In [7]:
camel_response_doc = return_camel_response_doc()

In [8]:
url_doc = return_url_doc(urls)

In [10]:
documents = camel_response_doc + url_doc

In [15]:
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings
from langchain_community.embeddings import HuggingFaceEmbeddings

In [13]:
embeddings = HuggingFaceEmbeddings()

c:\Users\Cypher\miniconda3\envs\rag\Lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
2024-06-04 18:04:44,688 - INFO - Use pytorch device_name: cpu
2024-06-04 18:04:44,689 - INFO - Load pretrained SentenceTransformer: sentence-transformers/all-mpnet-base-v2
c:\Users\Cypher\miniconda3\envs\rag\Lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [16]:
faiss_db = FAISS.from_documents(documents, embeddings)

2024-06-04 18:06:38,697 - INFO - Loading faiss with AVX2 support.
2024-06-04 18:06:38,783 - INFO - Successfully loaded faiss with AVX2 support.


In [ ]:
retriever = 